# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [72]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [18]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


A two sample hypothesis test using a z-statistic should be used to test whether or not the proportion of applicants being accepted from both groups is different or not. This dataset is large ebough for the CLT to apply as n is >= 30. The null and alternate hypothesis to be used for this test are:

H_o: There is no difference between the proportion of each population that got an interview; P1 - P2 = 0 

H_a: There is a difference between the proportion of each population that got an interview; P1 - P2 != 0

In [8]:
w = data[data.race=='w']
b = data[data.race=='b']

In [95]:
# Your solution to Q3 here
#Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

#p values

#bootstrapping b

#b1) Count size of each population and also the proportion of each population that recieved a call back
n_w = float(len(w.call))
n_b = float(len(b.call))

p_w = np.sum(w.call == 1) / n_w
p_b = np.sum(b.call == 1) / n_b

prop_diff_observed = p_w - p_b

#b3) Merge call data together to simulate H_0 that there is no difference between the populations

h0_pop = pd.concat([w.call,b.call])

#b4) Create a function to draw permutation replicates

def bs_perm_reps(data,size = 1):
    perm_reps = np.empty(size)
    
    for i in range(size):
            sample = np.random.permutation(data)
            n_bp = sample[:n_b]
            n_wp = sample[n_b:]
            
            p_bp = np.sum(n_bp == 1) / float(len(n_bp))
            p_wp = np.sum(n_wp == 1) / float(len(n_wp))
            
            perm_reps[i] = p_wp - p_bp
            
    return perm_reps

#b5) Bootstrap replicates and calculate the p-value of our observed proportion difference in the bootstrapped distribution

p_value_b = np.sum(bs_perm_reps(h0_pop, size = 10000) > prop_diff_observed) / float(10000)

print 'p_value_b: ', p_value_b

#Answer with bootstrapping. With a P-value of 0.0001 we can reject H_0 and say that the proportion of black and white people recieving call backs is not the same


#frequentist statistics f
#mean assuming h0
mean_h0 = 0

#std(p_w - p_b) = sqrt((1/n) * 2p(1-p)) Where p is the proportion of total call-backs in the entire population becasue h0 assumes p_w and p_b are the same
#std assuming h0
n_h0 = float(len(data.call))
p_h0 = np.sum(data.call == 1) / n_h0 
std_h0 = np.sqrt(2*p_h0*(1-p_h0)/ n_h0)

#z = (observed_proportion - mean_h0) / std_h0
z_h0 = (prop_diff_observed - mean_h0) / std_h0

p_value_f = stats.norm.sf(abs(z_h0))
print 'p_value_f:',p_value_f




# 95% confidence intervals and Margin of error for the difference in proportions of call backs
#for bernoulli distribution the average value of a data set is equal to the propability of success

mean_difference = prop_diff_observed

#std = sqrt((p_w(1-p_w) / n_w) - (p_b(1-p_b) / n_b))
std_combined = np.sqrt((p_w*(1-p_w)/n_w) - (p_b*(1-p_b)/n_b))

#z_scores necessary for 95 % confidence 
# use .975 in the function becuse it only returns one sided results or the percentage under the curve starting from the farthest left up until the value desired
z_score = stats.norm.ppf(.975)

Margin_of_Error_f = std_combined * z_score

print 'Confidence_Interval_f:', mean_difference, "+-", Margin_of_Error_f


def bootstrap_confidence_interval(data_w,data_b,size = 1,ci = .95):
    bs_reps = np.empty(size)
    
    for i in range(size):
        w_sample = np.random.choice(data_w, size = len(data_w))
        b_sample = np.random.choice(data_b, size = len(data_b))
        prop_diff = np.mean(w_sample) - np.mean(b_sample)
        
        
        bs_reps[i] = prop_diff
    
    std = np.std(bs_reps)
    mean = np.mean(bs_reps)
    ci = (1-ci)/2
    z_score = stats.norm.ppf(1-ci)
    margin_of_error = std * z_score 

    print 'Confidence_Interval_b:', mean, "+-", margin_of_error

    
bootstrap_confidence_interval(w.call,b.call,size = 1000)

        






p_value_b:  0.0
p_value_f: 3.12043108051e-09
Confidence_Interval_f: 0.0320328542094 +- 0.00651150123636
Confidence_Interval_b: 0.0319630391188 +- 0.0150401477643


C:\Users\Python\Miniconda2\envs\tutorial\lib\site-packages\ipykernel\__main__.py:28: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Python\Miniconda2\envs\tutorial\lib\site-packages\ipykernel\__main__.py:29: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future



There is concern that racial discrimination is preventing people of color from recieving work. To determine whether or not there were differences in the number of black applicants recieving interviews versus the number of white applicants recieving interviews we performed a two sample hypothesis test. For our null hypothesis, we assumed that there was no difference between the the proportion of applicants that were accepted between the black and white groups. Our resulting p-value from this test was practically 0. This means that our null hypothesis must be rejected and that there is some difference between the proportion of people being called back for interviews between the black and white data sets. However this does not necessarily mean that racial discrimination is the casue of this discrepency. Our test was to simply see if there was a difference between the two groups, not what the cause of the difference was. Since there are many more factors in play it is not possible to determine the significance of race with this test alone. To truly understand the affect of racial discrimination on the call back process we would need to control for experience, education, and age among other things. This could be accomplished by sending out resumes that are practically identical outside of race.


